# 識別率分布表
----  
  
引数：識別率-ボクセル-ブロードマンエリア対応表が記録されたcsvファイルがあるディレクトリまでのパス  
  
---  
  
入力：識別率-ボクセル-ブロードマンエリア対応表 ACCURACY_BA[交差検証法]_VOX(ベクトルの種類)_SVM.csv
   
---
  
出力：ACCURACY_DIST[交差検証法]_VOX(ベクトルの種類)_SVM.csv
  
---  
  
識別率の分布をまとめて書き出す  

In [1]:
from os.path import join as pathjoin
from pprint import pprint
# from nifti import NiftiImage
import glob
import numpy as np
import pandas as pd
import sys
import pickle
# import dill
import csv

# .pyで実行するときは%matplotlib inlineをコメントアウト！！！！
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
###### vecName,tsscan書き換え

# args = sys.argv
# PATH = args[1]

#jupyter notebookのときはここで指定
PATH = '../MaskBrodmann/'

# ベクトルの種類
vecName = 'univariate'

# 時系列幅
tsscan = '10'

if vecName != 'timeseries':
    
    tsscan = ''

Subject = ['20181227rn', '20181227tsk', '20181227tst']

# 交差検証法の種類
cv = '10cv'

# 機械学習アルゴリズム
ML = 'SVM'

# 各被験者ごとのRawDataディレクトリまでのパス

PATH_raw = []

for subName in Subject:
    
    PATH_raw = PATH_raw + [PATH + subName + '/RawData/']

In [11]:
for sub in PATH_raw:
    
    print(sub)
    
    ##### 各識別率ごとの分布 ###################
    
    
    # 識別率-ボクセル-ブロードマンエリア対応表csvファイルへのパス
    PATH_ac = sub + 'ACCURACY_BA' + '[' + cv + ']_VOX' + vecName + tsscan + '_' + ML + '.csv'

    # 読み込み
    table = pd.read_csv(PATH_ac, header = 0)

    # 識別率部分を取得
    ac = table['accuracy(%)']

    # 各識別率の出現頻度を一覧
    dist = pd.DataFrame(ac.value_counts())
    
    # 識別率の高い順にソート
    dist = dist.sort_index(ascending = False)
    
    # 各識別率がインデックスになっているので解除，列名をつける
    dist = dist.reset_index()
    dist.columns = ['accuracy(%)', 'Number']


    PATH_csv = sub + 'ACCURACY_DIST[' + cv + ']_VOX' + vecName + tsscan + '_' + ML + '.csv'
    print(PATH_csv)
    dist.to_csv(PATH_csv)
    
    
    ##### ある区間で区切った時の分布 ####################
    
    # 個数格納用
    acList = []
    
    # インデックス用
    acList_index = []

    # sepNum%区切り
    sepNum = 10

    for i in range(sepNum):

        # 識別率の区間を指定
        mask = ((dist['accuracy(%)'] >= (i * sepNum)) & (dist['accuracy(%)'] < ((i + 1) * sepNum)))
        distSep = dist[mask]

        # 区間内のボクセル数をカウント
        acList = acList + [distSep['Number'].sum()]

        # インデックス名作成
        if i == 9:

            indexName = [str(i*10) + ' ~ ' + str(((i + 1) * 10))]

        else:

            indexName = [str(i*10) + ' ~ ' + str(((i + 1) * 10) - 1)]

        acList_index = acList_index + indexName

    # データフレーム化，インデックス・カラム名追加
    acList = pd.DataFrame(acList)
    acList.index = acList_index
    acList.columns = ['Distribution']
    
    # 識別率の高い順にソート
    acList = acList.sort_index(ascending = False)

    PATH_csv = sub + 'ACCURACY_DIST' + str(sepNum) + '[' + cv + ']_VOX' + vecName + tsscan + '_' + ML + '.csv'
    print(PATH_csv)
    acList.to_csv(PATH_csv)
    

../MaskBrodmann/20181227rn/RawData/
../MaskBrodmann/20181227rn/RawData/ACCURACY_DIST[10cv]_VOXunivariate_SVM.csv
../MaskBrodmann/20181227rn/RawData/ACCURACY_DIST10[10cv]_VOXunivariate_SVM.csv
../MaskBrodmann/20181227tsk/RawData/
../MaskBrodmann/20181227tsk/RawData/ACCURACY_DIST[10cv]_VOXunivariate_SVM.csv
../MaskBrodmann/20181227tsk/RawData/ACCURACY_DIST10[10cv]_VOXunivariate_SVM.csv
../MaskBrodmann/20181227tst/RawData/
../MaskBrodmann/20181227tst/RawData/ACCURACY_DIST[10cv]_VOXunivariate_SVM.csv
../MaskBrodmann/20181227tst/RawData/ACCURACY_DIST10[10cv]_VOXunivariate_SVM.csv
